In [89]:
import datetime as dt
import numpy as np
import os
import pandas as pd

from dashboard_app.common.models.reporter_model import ReporterModel
from dashboard_app.common.models.read_file_model import ReadExlFileModel
from dashboard_app.common.models.calc_model import Calc

headless_mode = True
url = 'http://sccts7dxsql/ctreport'
id = 'eisuke_koresawa'

In [90]:
# メンバリストファイル
member_list_path = r"\\mjs.co.jp\datas\CSC共有フォルダ\第47期 東京CSC第二グループ\47期SV共有\ダッシュボードアイテム"
member_list_file = 'メンバーリスト.xlsx'
member_list_file_path = os.path.join(member_list_path, member_list_file)

# 'クローズ_本日.xlsm'ファイル
close_file = r'\\mjs.co.jp\datas\CSC共有フォルダ\第47期 東京CSC第二グループ\47期SV共有\ダッシュボードアイテム\クローズ_本日.xlsm'

In [91]:
# レポータ、ファイル読込み、計算インスタンスを作成
rm = ReporterModel(url, id, headless_mode=headless_mode)
rf = ReadExlFileModel()
cl = Calc()

In [92]:
# レポータから本日の各種情報を読込みDataFrameへ格納
df = rm.get_today_op_as_dataframe()

In [93]:
# メンバーリストファイルから、レポータの名前を正式な氏名のフォーマットに変換
member_list = pd.read_excel(member_list_file_path)
index_dict_reporter = member_list.set_index('レポータ')['氏名'].to_dict()
df.index = df.index.map(index_dict_reporter)

In [94]:
# クローズ_本日.xlsmファイルを読み込み、クローズのDataFrameを作成
df_close = rf.read_today_close_file()

In [95]:
# シフトデータを読込み、同時にメンバーリストファイルから、名前を正式な氏名のフォーマットに変換
df_shift = rf.read_today_shift_file()
index_dict_shift_table = member_list.set_index('Sweet')['氏名'].to_dict()
df_shift.index = df_shift.index.map(index_dict_shift_table)

In [96]:
# 当番データを読込み、同時にメンバーリストファイルから、名前を正式な氏名のフォーマットに変換
df_duty = rf.read_today_duty_file()
index_dict_duty_table = member_list.set_index('Sweet')['氏名'].to_dict()
df_duty.index = df_duty.index.map(index_dict_duty_table)

In [97]:
# クローズデータとレポータからスクレイピングしたデータをjoin
df_join = df.join(df_close, how='outer').fillna(0)

In [98]:
df_join# 各カラムの合計値を計算して新しい行として追加
df_join.loc['合計'] = df_join.sum()

In [99]:
# シフトデータをjoin
dataset = df_join.join(df_shift, how='outer').fillna('未設定')
dataset = df_join.join(df_duty, how='outer').fillna('')

In [100]:
dataset['ACW'] = (dataset['ワークタイムの合計'] + dataset['着信後処理時間の合計'] + dataset['発信後処理時間の合計'] + dataset['事前準備時間の合計'] + dataset['転送可時間の合計'] + dataset['一時離席時間の合計']) / dataset['ｸﾛｰｽﾞ']
dataset['ATT'] = (dataset['着信通話時間の合計(外線)'] + dataset['発信通話時間の合計(外線)']) / dataset['ｸﾛｰｽﾞ']
_tmp = dataset['着信通話時間の合計'] + dataset['発信通話時間の合計'] + dataset['ワークタイムの合計'] + dataset['着信後処理時間の合計'] + dataset['発信後処理時間の合計'] + dataset['離席時間の合計'] + dataset['事前準備時間の合計'] + dataset['一時離席時間の合計']
dataset['CPH'] = dataset['ｸﾛｰｽﾞ'] / (_tmp * 24)

In [101]:
dataset = dataset.replace(np.inf, 0)

In [103]:
dataset

,ログオン時間,着信通話時間の合計,着信通話時間の合計(外線),発信通話時間の合計,発信通話時間の合計(外線),ワークタイムの合計,着信後処理時間の合計,発信後処理時間の合計,離席時間の合計,待機時間,...,別作業中時間の合計,転送可時間の合計,他者支援時間の合計,開発資料確認時間の合計,資料作成時間の合計,ｸﾛｰｽﾞ,業務ｽﾃｰﾀｽ,ACW,ATT,CPH
三ケ田 梨乃,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,,NaN,NaN,NaN
伊藤 克泰,0.000000,0.004861,0.002431,0.011701,0.011701,0.015949,0.020984,0.004826,0.0,0.000706,...,0.005729,0.0,0.000000,0.0,0.0,2.0,,0.027315,0.007066,1.170541
内御堂 吉彦,0.000000,0.000000,0.000000,0.038611,0.038611,0.031620,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,5.0,,0.006324,0.007722,2.966381
北村 聡洋,0.309444,0.031563,0.031563,0.001134,0.001134,0.006238,0.010984,0.000000,0.0,0.158877,...,0.000000,0.0,0.000000,0.0,0.0,7.0,,0.003388,0.004671,5.170291
原口 順子,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,,NaN,NaN,NaN
合計,0.354120,0.225521,0.157546,0.259352,0.259352,0.346481,0.145324,0.070868,0.0,0.403264,...,0.222512,0.0,0.006088,0.0,0.0,83.0,,0.008179,0.005023,2.971685
吉岡 崇志,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,,NaN,NaN,NaN
吉本 優成,0.000000,0.002384,0.002384,0.009282,0.009282,0.017384,0.009213,0.000000,0.0,0.000012,...,0.000000,0.0,0.006088,0.0,0.0,4.0,,0.008458,0.002917,3.663190
吉田 祐二,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,SV,NaN,NaN,NaN
大竹 航平,0.000000,0.000000,0.000000,0.000000,0.000000,0.000058,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,,0.000000,NaN,0.000000


In [102]:
# 必要なカラムのみ抽出
dataset = dataset[['ｼﾌﾄ', 'ACW', 'ATT', 'CPH', 'ｸﾛｰｽﾞ', '業務ｽﾃｰﾀｽ']]

KeyError: "['ｼﾌﾄ'] not in index"